In [2]:
import math
import random

In [3]:
class Value:
    
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self._prev = set(_children)
        self._op = _op
        self.grad = 0.0
        self._backward = lambda: None
        
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data**(other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): # other * self
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + -other
    
    def __radd__(self, other): # other + self
        return self + other

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1) 
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward
        
        return out
    
    def backward(self):
        
        topo = []
        visited = set()
        
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()    

In [4]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1, 1))
        
    def __call__(self, x):
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        
        return out
    
    def parameters(self):
        return self.w + [self.b]
    
class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]
    
class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [5]:
n = MLP(3, [4,4,1])

xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]

ys = [1.0, -1.0, -1.0, 1.0]

In [6]:
for epochs in range(1000):
    
    ypred = [n(x) for x in xs]
    loss = sum((y_hat - ygt)**2 for ygt, y_hat in zip(ys, ypred))
    
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    for p in n.parameters():
        p.data += -0.01 * p.grad
    
    print(epochs, ":", loss.data)

0 : 3.3546754257767892
1 : 2.927181456344758
2 : 2.560452558991485
3 : 2.2516869978352876
4 : 1.9920994952493483
5 : 1.7719973901349029
6 : 1.5834646324532202
7 : 1.4207533741711096
8 : 1.2796917727025643
9 : 1.157065030548083
10 : 1.050237817488243
11 : 0.9569701021221582
12 : 0.8753332575439801
13 : 0.8036664994369043
14 : 0.7405461095063406
15 : 0.6847577752979056
16 : 0.6352699394174903
17 : 0.591208488787172
18 : 0.5518335034660227
19 : 0.516518545064443
20 : 0.4847326432866122
21 : 0.4560248987096198
22 : 0.4300114800017276
23 : 0.40636473158132297
24 : 0.38480409541476757
25 : 0.36508856635375275
26 : 0.34701042947908717
27 : 0.33039006148999484
28 : 0.3150716113536171
29 : 0.3009194057872292
30 : 0.28781495172222726
31 : 0.27565443053342903
32 : 0.26434659775419833
33 : 0.25381101765740693
34 : 0.2439765749334139
35 : 0.23478021619229927
36 : 0.2261658825662569
37 : 0.21808360164135193
38 : 0.2104887126019684
39 : 0.20334120307144113
40 : 0.19660513987916778
41 : 0.190248179041

In [7]:
ypred

[Value(data=0.9748771162629278),
 Value(data=-0.9629117620670941),
 Value(data=-0.9724613646428895),
 Value(data=0.9677359904289965)]